<a href="https://colab.research.google.com/github/Kishan-Kumar-Zalavadia/Agentic_AI_with_Python_And_Generative_AI/blob/main/2_AIAgentsAndGenerativeAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI-Agent Tool Descriptions and Naming**

## **Describing Tools to the Agent**

When developing an agentic AI system, one of the most critical aspects is ensuring that the agent understands the tools it has access to. In our previous tutorial, we explored how an AI agent interacts with an environment. Now, we extend that discussion to focus on tool definition, particularly the importance of naming, parameters, and structured metadata.

## **Example: Automating Documentation for Python Code**

Imagine we are building an AI agent that scans through all Python files in a src/ directory and automatically generates corresponding documentation files in a docs/ directory. This agent will need to:

- List Python files in the src/ directory.
- Read the content of each Python file.
- Write documentation files in the docs/ directory.

Since file operations are straightforward for humans but ambiguous for an AI without context, we must clearly define these tools so the agent knows how to use them effectively.


# *Step 1: Defining a Tool with Structured Metadata*

A basic tool definition in Python might look like this:

```
def list_python_files():
    """Returns a list of all Python files in the src/ directory."""
    return [f for f in os.listdir("src") if f.endswith(".py")]
```

This provides a function that retrieves all Python files in the src/ directory, but for an AI system, we need a more structured way to describe it.

# **tSep 2: Using JSON Schema to Define Parameters**

When developers design APIs, they use structured documentation to describe available functions, their inputs, and their outputs. JSON Schema is a well-known format for defining APIs, making it a natural choice for AI agents as well.

For example, a tool that reads a file should specify that it expects a file_path parameter of type string. JSON Schema allows us to express this in a standardized way:

```
{
  "tool_name": "read_file",
  "description": "Reads the content of a specified file.",
  "parameters": {
    "type": "object",
    "properties": {
      "file_path": { "type": "string" }
    },
    "required": ["file_path"]
  }
}
```

Similarly, a tool for writing documentation should define that it requires a file_name and content:



```
{
  "tool_name": "write_doc_file",
  "description": "Writes a documentation file to the docs/ directory.",
  "parameters": {
    "type": "object",
    "properties": {
      "file_name": { "type": "string" },
      "content": { "type": "string" }
    },
    "required": ["file_name", "content"]
  }
}
```

By providing a JSON Schema for each tool:

1. The AI can Recognize the tool’s purpose.
2. The AI / Environment interface can validate input parameters before execution.

It may look strange that multiple parameters to a function are represented as an object. When we are getting the agent to output a tool / action selection, we are going to want it to output something like this:

```
{
  "tool_name": "read_file",
  "args": {
    "file_path": "src/file.py"
  }
}
```

The schema describes the overall dictionary that will be used to capture the “args” to the function, so it is described as an object.



# Install libraries

In [ ]:
!!pip install litellm
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

# Agent that Calls Python Functions

Let's make an agent that can call Python functions.
When you run the second block in the notebook, the agent will prompt you for what action to take. You can say something like "tell me the files in the current directory" and it will make the appropriate tool choice. Experiment with the agent to see what it can and cannot do.

In [8]:
import json
import os
import sys
from litellm import completion
from typing import List, Dict

def extract_markdown_block(response: str, block_type: str = "json") -> str:
    """Extract code block from response"""

    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()

    if code_block.startswith(block_type):
        code_block = code_block[len(block_type):].strip()

    return code_block

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get a response."""
    response = completion(
        model="groq/llama-3.3-70b-versatile",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content.strip()

def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

# Define system instructions (Agent Rules)
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:

```json
{
    "list_files": {
        "description": "Lists all files in the current directory.",
        "parameters": {}
    },
    "read_file": {
        "description": "Reads the content of a file.",
        "parameters": {
            "file_name": {
                "type": "string",
                "description": "The name of the file to read."
            }
        }
    },
    "terminate": {
        "description": "Ends the agent loop and provides a summary of the task.",
        "parameters": {
            "message": {
                "type": "string",
                "description": "Summary message to return to the user."
            }
        }
    }
}
```

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.

Important!!! Every response MUST have an action.
You must ALWAYS respond in this format:

<Stop and think step by step. Parameters map to args. Insert a rich description of your step by step thoughts here.>

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```"""
}]

# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

# The Agent Loop
while iterations < max_iterations:
    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)
    result = "Action executed"

    if action["tool_name"] == "list_files":
        result = {"result": list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break

    iterations += 1


What would you like me to do? Tell me what is in the dir.
Agent thinking...
Agent response: To find out what is in the current directory, I need to list all the files. This will give me a clear understanding of the files available before I can proceed with any further actions. By listing the files, I can identify the types of documents or content available and provide a more accurate response to the user's inquiry.

```action
{
    "tool_name": "list_files",
    "args": {}
}
```
Action result: {'result': ['.config', 'Readme.txt', '.ipynb_checkpoints', 'sample_data']}
Agent thinking...
Agent response: The directory contains several files and directories, including ".config", "Readme.txt", ".ipynb_checkpoints", and "sample_data". Since the user initially asked about the contents of the directory, I should now read the "Readme.txt" file to provide more information about the directory's contents. The "Readme.txt" file often contains important information or descriptions about the directory